# LangChain 核心模块 Agent - Self ask with search

- Google 搜索对接
    - 第三方平台：https://serpapi.com
    - LangChain API 封装：`SerpAPIWrapper`
- LangChain Agent 模块使用
    - Agent 类型：`SELF_ASK_WITH_SEARCH`
    - Agent 实例化：`initialize_agent` 方法
 
https://python.langchain.com/docs/modules/agents/agent_types/

In [1]:
import os

# 更换为自己的 Serp API KEY
# os.environ["SERPAPI_API_KEY"] = "1e3c93e0753ac224098370cd71da86150c6609caba6b5aaa307e04b72a5006e1"
os.environ["SERPAPI_API_KEY"] = "f805ac0b9ece2ec2c2ba22584afed0d6fc881a86cf541c39962ecdb9ea330d25"

In [15]:
from langchain_openai import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = ChatOpenAI(temperature=0, model="gpt-4")

In [3]:
# 实例化查询工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [7]:
# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
     tools,llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [16]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search.invoke(
    "成都举办的大运会是第几届大运会？"
)



> Entering new AgentExecutor chain...
Yes.

Follow up: 成都举办的大运会是哪一年举办的？
Intermediate answer: 大运会所有必选竞赛项目全部是奥运项目，竞赛管理由国际单项体育组织按照奥运标准实施，是规模仅次于奥运会的世界综合性运动会，素有“小奥运会”之称。 2023年7月28日至8月8日，第31届世界大学生夏季运动会在四川成都举办。
Intermediate answer: 2023年7月28日至8月8日，第31届世界大学生夏季运动会在四川成都举办。
So the final answer is: 第31届

> Finished chain.


{'input': '成都举办的大运会是第几届大运会？', 'output': '第31届'}

In [17]:
# 实际运行 Agent，查询问题（有歧义）
self_ask_with_search.run(
    "2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
No.

So the final answer is: 成都, 中国 (Chengdu, China)

> Finished chain.


'成都, 中国 (Chengdu, China)'

In [18]:
# Reason-only 答案（错误）
print(llm.invoke("成都举办的大运会是第几届大运会？"))

content='成都举办的是第31届大运会。' response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 27, 'total_tokens': 43}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-3f9c4420-f5f0-401c-87f9-2faf1ccad0d4-0'


In [19]:
# Reason-only 答案（错误）
print(llm.invoke("2023年大运会举办地在哪里？"))

content='2023年大运会的举办地在中国的成都市。' response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 23, 'total_tokens': 42}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-f130e954-8511-467c-afb7-4467b30d2cd6-0'


#### 使用 GPT-4 作为大语言模型实现更优的 ReAct 范式

In [10]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

In [11]:
self_ask_with_search_chat = initialize_agent(
    tools, chat_model, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [ ]:
# GPT-4 based ReAct 答案（正确）
self_ask_with_search_chat.run(
    "2023年大运会举办地在哪里？"
)

In [13]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search_chat.invoke(
    "成都举办的大运会是第几届大运会？"
)



> Entering new AgentExecutor chain...
Yes.
Follow up: 成都举办的大运会是哪一年的？
Intermediate answer: 2021年夏季世界大学生运动会（英语：2021 Summer World University Games）简称为2021年成都大运会或成都大运会，是第31届夏季世界大学生运动会，原定于2021年于中国四川省成都市举行，后因2019冠状病毒病疫情等原因二次延期，最终改至2023年7月28日至8月8日举行。
So the final answer is: 第31届大运会

> Finished chain.


{'input': '成都举办的大运会是第几届大运会？', 'output': '第31届大运会'}